## Setup

In [1]:
import numpy as np
import pandas as pd

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_days_to_pref_cost, cal_total_preference, cal_total_accounting
)
from util_check import deep_check

In [2]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
path_init_conf =     '../output/m08-improved.csv'
path_dump_improved = '../output/m11-improved.csv' # lowest cost
num_cpu_cores = 6
#time_limit = 10*60*60*1000  # in ms
time_limit = 60*1000  # in ms

In [3]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

Read initial configs...
Read config completed.
Init config cost: 73610.22770050484


In [4]:
len(assigned_day)

5000

In [5]:
families = range(N_families)
days = range(1, N_days + 1)

In [6]:
allowed_occupancy = range(N_min_people, N_max_people+1)

## Ortools - CBC MIP solver

In [7]:
from ortools.linear_solver import pywraplp

In [8]:
solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [9]:
print('Set num threads:', solver.SetNumThreads(num_cpu_cores))
print('Set time limit:', solver.SetTimeLimit(time_limit))

Set num threads: True
Set time limit: None


## Variables

In [10]:
# Variables
assignment_matrix = {}
for family in families:
    for day in days:
        assignment_matrix[family, day] = solver.BoolVar('x[%i,%i]' % (family, day))

In [11]:
len(assignment_matrix)

500000

In [12]:
# one-hot encoding of occupancy of each day
occupancy_matrix = {}
for day in days:
    for people in allowed_occupancy:
        occupancy_matrix[day, people] = solver.BoolVar('o[%i,%i]' % (day, people))

In [13]:
len(occupancy_matrix)

17600

## Constraints

In [14]:
# Constraints
# each family only take 1 day
for family in families:
    solver.Add(
        solver.Sum([assignment_matrix[family, day] for day in days]) == 1
    )

In [15]:
# each day can only have 1 occupancy number
for day in days:
    solver.Add(
        solver.Sum([occupancy_matrix[day, people] for people in allowed_occupancy]) == 1
    )

In [16]:
for day in days:
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, day] * n_people[family] for family in families]
        ) <= N_max_people
    )
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, day] * n_people[family] for family in families]
        ) >= N_min_people
    )

In [17]:
for day in days:
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, day] * n_people[family] for family in families]
        )
        == solver.Sum(
            [occupancy_matrix[day, people] * people for people in allowed_occupancy]
        )
    )

In [18]:
len(solver.constraints())

5400

## Objective

In [19]:
# Objective - Preference cost only as approximation
solver.Minimize(
    solver.Sum([
        assignment_matrix[family, day] * family_id_days_to_pref_cost[family, day]
        for family in families for day in days
    ])
)

## Solve

In [20]:
print('N of variables', len(solver.variables()))

N of variables 517600


In [21]:
%%time
# Solve
sol = solver.Solve()

resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
           4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
print('Result: ', resdict[sol])
print('Total cost: ', solver.Objective().Value())
print("Time: ", solver.WallTime(), " milliseconds")

Result:  NOT_SOLVED
Total cost:  0.0
Time:  122550  milliseconds
CPU times: user 1min 30s, sys: 6.43 s, total: 1min 36s
Wall time: 1min 37s


## Solution

In [22]:
assigned_day_new = np.array([
    [0]+[assignment_matrix[family, day].solution_value() for day in days]
    for family in families
]).argmax(axis=1)

In [23]:
assigned_day_new.max()

0

In [24]:
family_on_day_new = assigned_day_to_family_on_day(assigned_day_new)

occupancy_new = assigned_day_to_occupancy(assigned_day_new)

deep_check(assigned_day_new, family_on_day_new, occupancy_new)

deep check: everything looks fine.


True

In [25]:
print('Total score:    ', cal_total(assigned_day_new, occupancy_new))
print('Preference cost:', cal_total_preference(assigned_day_new))
print('Accounting cost:', cal_total_accounting(occupancy_new))

Total score:     499995000.0
Preference cost: 499995000
Accounting cost: 0.0


## Output

In [26]:
dump_conf(assigned_day_new, path_dump_improved)